### 부산시 주유소 유가 데이터 전처리
#### 기간 : 20220214~20220809
#### 주차별 평균으로 사용

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import folium
import re

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from wordcloud import WordCloud

from datetime import datetime
import time

In [ ]:
font_path = 'C:/windows/fonts/NanumBarunpenB.ttf'
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

In [ ]:
# 글자 선명
from IPython.display import set_matplotlib_formats

set_matplotlib_formats("retina")

In [ ]:
#데이터 불러오기 
data = pd.read_csv('./data/부산시 주유소 유가 데이터(20220209-20220809).csv', encoding='utf-8', sep=',')
data.head()

#### 데이터 확인

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data.describe()

In [ ]:
#휘발유 0값 추출
data[data["휘발유"] == 0].head()

In [ ]:
#경유 0값 추출
len(data[data["경유"] == 0])

#### 데이터 전처리 시작

##### 0 -> NaN

In [ ]:
data['휘발유'] = data['휘발유'].replace(0,np.NaN)
data['경유'] = data['경유'].replace(0,np.NaN)

#### 경유 정리
#### 경유 NaN 추출

In [ ]:
# 전체 내용을 보기 위해서 max_rows 설정
pd.set_option('display.max_rows',250)

In [ ]:
data[data['경유'].isnull()]

#### 결측치 채워넣기
#### 경유는 결측치 별로 없어서 전체 주유소의 특정 기간 앞,뒤 유가 비교 후 채우기
- 4월에 결측치가 많다.
    - 20220401을 대표로 앞, 뒤 비교 후 어떻게 채울지 결정

In [ ]:
data[(data['기간']==20220330)|(data['기간']==20220331)|(data['기간']==20220401)|(data['기간']==20220402)|(data['기간']==20220403)].head(40)

##### 대체로 앞의 기간과 유가가 동일하다고 보여지므로 앞의 유가로 NaN 채우기

In [ ]:
data['경유']=data['경유'].fillna(method='ffill')

In [ ]:
data[data['경유'].isnull()]

#### 휘발유 정리
#### 휘발유 NaN 추출

In [ ]:
data[data['휘발유'].isnull()]

#### 결측치 확인
#### 굿오일주유소: 휘발유값 전체날짜에 NaN -> 경유만 파는 곳임을 확인
- 부산시 주유소 나타낼 때는 필요
- 주유소별 가격비교할 때는 필요X
    - ~contains

In [ ]:
data = data[~data['상호'].str.contains('굿오일주유소')]

#### (주)동일주유소: 2월 휘발유값 없다.
- (주)동일주유소의 경유값이 앞의 날짜와 동일하게 책정되어 있으므로 휘발유값도 앞의 날짜로 채우기

In [ ]:
data[data['상호']=='(주)동일주유소'].head(30)

#### 그 외 주유소 확인
- 경유와 마찬가지로 4월에 누락이 많아서 대표로 확인하기

In [ ]:
data[(data['기간']==20220330)|(data['기간']==20220331)|(data['기간']==20220401)|(data['기간']==20220402)|(data['기간']==20220403)].head(40)

#### 결측치 채워넣기
#### 경유와 마찬가지로 앞의 기간과 유가가 동일하다고 보여지므로 앞의 유가로 NaN 채우기

In [ ]:
data['휘발유']=data['휘발유'].fillna(method='ffill')

In [ ]:
data[data['휘발유'].isnull()]

#### 최종 결과 확인

In [ ]:
data.info()

#### 결측치를 유가로 채워넣은 내용 새 파일에 저장하기

In [ ]:
data.to_csv('./data/부산시유가데이터_fillna.csv', index=False, encoding='utf-8')

#### 연도별 주차 평균 데이터

In [ ]:
#int64->object->datetime 타입변경
data['기간'] = pd.to_datetime(data['기간'].astype(str))
data.info()

In [ ]:
#연도별 주차 컬럼 생성
data["주차"] = data["기간"].dt.isocalendar().week

In [ ]:
data.head(10)

#### 주차별 휘발유/경유 평균 가격 비교

In [ ]:
#주차별 휘발유 평균 가격(반올림, int로 변경)
weekly_avg_oil = data.groupby(["주차"])["휘발유","경유"].mean().round().astype("int")
weekly_avg_oil

In [ ]:
#컬럼명 변경
weekly_avg_oil.rename(columns={"휘발유" : "휘발유_평균가격", "경유" : "경유_평균가격"}, inplace=True)
weekly_avg_oil

#### 주차별 평균가격 변화 추세 시각화

In [ ]:
from turtle import width


ax1 = weekly_avg_oil.index
ax2 = weekly_avg_oil.휘발유_평균가격
ax3 = weekly_avg_oil.경유_평균가격

plt.figure(figsize=(40,10))
a = plt.plot(ax1, ax2, label='휘발유 평균 가격', linestyle="-.", color='Hotpink', linewidth=5, alpha=0.5) # 휘발유 평균 가격
b = plt.plot(ax1, ax3, label='경유 평균 가격', linestyle=":", color='green', linewidth=5, alpha=0.5) # 경유 평균 가격
p = [a,b]

plt.title("2022년 주차별 휘발유/경유 평균가격 추세", fontsize=40)
plt.xticks(ax1, fontsize=20, labels=["2월 3째주","2월 4째주",
                                     "3월 1째주","3월 2째주","3월 3째주","3월 4째주","3월 5째주",
                                     "4월 1째주","4월 2째주","4월 3째주","4월 4째주",
                                     "5월 1째주","5월 2째주","5월 3째주","5월 4째주",
                                     "6월 1째주","6월 2째주","6월 3째주","6월 4째주","6월 5째주",
                                     "7월 1째주","7월 2째주","7월 3째주","7월 4째주",
                                     "8월 1째주","8월 2째주"], rotation=45) # x축
plt.text(11.8,2015,'휘발유 평균 가격',fontsize=30) # 그래프 이름
plt.text(12,1850,'경유 평균 가격',fontsize=30)
plt.yticks(np.arange(1500, 2300, 25) ,fontsize=15) # y축
plt.xlabel("주차(weeks)", fontsize=25) #x 축 이름
plt.ylabel("가격(원)", fontsize=25) # y축 이름
#plt.legend(loc="best",ncol=2, fontsize= 20, shadow=True) # 범례
plt.grid()

plt.show()

#### 상표별 휘발유/경유 평균 가격 비교

In [ ]:
#상표별 휘발유/경유 평균가격(반올림, int로 변경)
brand_avg_oil = data.groupby(["상표"])["휘발유","경유"].mean().round().astype("int")
brand_avg_oil

In [ ]:
#컬럼명 변경
brand_avg_oil.rename(columns={"휘발유" : "휘발유_평균가격", "경유" : "경유_평균가격"}, inplace=True)
brand_avg_oil

#### 상표별 평균 가격 비교(22년 2월14일 ~ 8월 9일) 시각화

In [ ]:
w = 0.28 #widths
nrow = brand_avg_oil.shape[0] #행의 갯수
idx = np.arange(nrow) #행의 갯수를 리스트로
idx


plt.figure(figsize=(20,10))
plt.bar(idx-w, brand_avg_oil["휘발유_평균가격"], width=w, align="edge", color="yellow", alpha=0.5, hatch='+')
plt.bar(idx+w-0.14, brand_avg_oil["경유_평균가격"], width=w, color="green", alpha=0.5, hatch='x')
plt.title("상표별 유가 비교 그래프", fontsize=35)
plt.xticks(idx, brand_avg_oil.index, fontsize=15) # x축
plt.yticks(np.arange(0,2300, 100), fontsize=15) # y축
plt.xlabel("상표명", fontsize=15) #x 축 이름
plt.ylabel("가격(원)", fontsize=15) # y축 이름
plt.legend(brand_avg_oil.columns, loc="best", ncol=nrow, fontsize= 15, shadow=True) # 범례
plt.grid(alpha=0.3)

for idx, value in enumerate(list(brand_avg_oil["휘발유_평균가격"])): # 휘발유 가격 표시
        txt = '%d원' % value
        plt.text(idx,value,txt,horizontalalignment='right',
                verticalalignment='bottom', fontsize= 13)

for idx, value in enumerate(list(brand_avg_oil["경유_평균가격"])): # 경유 가격 표시
        txt = '%d원' % value
        plt.text(idx,value,txt,horizontalalignment='left',
                verticalalignment='bottom', fontsize= 13)


plt.show()

#### 가격 변동 추이 영상화

!pip install bar_chart_race

!pip install ffmpeg

In [ ]:

import bar_chart_race as bcr
from matplotlib.animation import FuncAnimation
import matplotlib.ticker as ticker
import matplotlib.animation as animation
from IPython.display import HTML
import ffmpeg

## 전처리

In [ ]:
data.info()

In [ ]:
data['기간'] = pd.to_datetime(data['기간'], format='%Y%m%d')

In [ ]:
data = data[['기간','상표', '셀프여부', '휘발유', '경유']]
data.head()

In [ ]:
data["상표셀프여부"] = data["상표"] + data["셀프여부"]
data.head()

In [ ]:
data = data.drop(data.columns[[1, 2]], axis=1) 
data.tail()

In [ ]:
data_diesel = data.drop(data.columns[[1]], axis=1) 
data_diesel.tail(5)

In [ ]:
data_gasolin = data.drop(data.columns[[2]], axis=1) 
data_gasolin.tail(5)

In [ ]:
data_gasolin = data_gasolin.pivot_table(values = '휘발유', index = ['기간'], columns = '상표셀프여부')
data_gasolin.tail()

In [ ]:
data_diesel = data_diesel.pivot_table(values = '경유', index = ['기간'], columns = '상표셀프여부')
data_diesel.tail()

In [ ]:
data_diesel.columns[0]

In [ ]:
data_diesel.index


### bar_chart_race 을 이용한 가격 변동 추이 영상 만들기

In [ ]:
bcr.bar_chart_race(data_gasolin,
                    n_bars=14,
                    figsize=(6,4),
                    sort='desc',
                    title='휘발유 가격 변동 추이',
                    fixed_max= True,
                    period_fmt='%B %d, %Y',
                    filename='./data/gasolin_price.mp4',
                    
                    perpendicular_bar_func='median'
                    )

In [ ]:
bcr.bar_chart_race(data_diesel,
                    n_bars=14,
                    figsize=(6,4),
                    sort='desc',
                    title='경유 가격 변동 추이',
                    fixed_max= True,
                    period_fmt='%B %d, %Y',
                    filename='./data/disel_price.mp4',
                    
                    perpendicular_bar_func='median'
                    )


#### 부산시 구별 월평균 휘발유 가격 비교

In [ ]:
data = pd.read_csv('./data/부산시유가데이터_fillna.csv', encoding='utf-8')
data

In [ ]:
#int64->object->datetime 타입변경
data['기간'] = pd.to_datetime(data['기간'].astype(str))
data.info()

In [ ]:
#연도별 주차 컬럼 생성
data["주차"] = data["기간"].dt.isocalendar().week

In [ ]:
#월(month) 열 추가
data["월"] = data["기간"].dt.month 
data.sample(5)

In [ ]:
#지역별, 월별 휘발유/경유 평균 가격(반올림, int로 변경)
gu_monthly_oil_avg_price = data.groupby(["지역","월"])["휘발유","경유"].mean().round().astype("int")
gu_monthly_oil_avg_price

In [ ]:
gu_monthly_oil_max_price = gu_monthly_oil_avg_price.groupby(["지역"])[["휘발유","경유"]].max()
gu_monthly_oil_max_price

#### 부산시 구별 휘발유/경유 월평균 가격 시각화

In [ ]:
w = 0.28 #widths
nrow = gu_monthly_oil_max_price.shape[0] #행의 갯수
idx = np.arange(nrow) #행의 갯수를 리스트로
idx


plt.figure(figsize=(40,20))
plt.bar(idx-w, gu_monthly_oil_max_price["휘발유"], width=w, align="edge", color="yellow", alpha=0.5)
plt.bar(idx+w-0.14, gu_monthly_oil_max_price["경유"], width=w, color="green", alpha=0.5)
plt.title("행정구별 월평균 유가 비교 그래프", fontsize=35)
plt.xticks(idx, gu_monthly_oil_max_price.index, fontsize=15) # x축
plt.yticks(np.arange(0,2400, 100), fontsize=15) # y축
plt.xlabel("행정구", fontsize=15) #x 축 이름
plt.ylabel("가격(원)", fontsize=15) # y축 이름
plt.legend(gu_monthly_oil_max_price.columns, loc="best", ncol=nrow, fontsize= 20, shadow=True) # 범례
plt.grid(alpha=0.3)

for idx, value in enumerate(list(gu_monthly_oil_max_price["휘발유"])): # 휘발유 가격 표시
        txt = '%d원' % value
        plt.text(idx,value,txt,horizontalalignment='right',
                verticalalignment='bottom', fontsize= 15)

for idx, value in enumerate(list(gu_monthly_oil_max_price["경유"])): # 경유 가격 표시
        txt = '%d원' % value
        plt.text(idx,value,txt,horizontalalignment='left',
                verticalalignment='bottom', fontsize= 15)

plt.show()

In [ ]:
import json
import folium

In [ ]:
path = 'C:\localRepository\Busan-Oil-Price-Analysis\data\행정구별.geojson'

In [ ]:
rfile = open(path,'r',encoding='utf8').read()
jsonfile = json.loads(rfile)

json_busan = {'type':'FeatureCollection'}
json_pick = []
json_gu = []

for item in jsonfile['features']:
    gu = item['properties']['SIG_ENG_NM']
    item['id'] = gu
    json_gu.append(gu)
    json_pick.append(item)

json_busan['features'] = json_pick

In [ ]:
gu_monthly_oil_max_price['ENG_NM'] = ['Gangseo-gu', 'Geumjeong-gu', 'Gijang-gun', 'Nam-gu', 'Dong-gu', 'Dongnae-gu', 'jin-gu', 'Buk-gu', 'Sasang-gu', 'Saha-gu', 'Seo-gu', 'Suyeong-gu', 'Yeonje-gu', 'Yeongdo-gu', 'Jung-gu', 'Haeundae-gu']
gu_monthly_oil_max_price

In [ ]:
map = folium.Map((35.1795543,129.0756416), zoom_start=10)
choropleth = folium.Choropleth(json_busan,
                  data=gu_monthly_oil_max_price,
                  columns=['ENG_NM', '휘발유'],
                  key_on='feature.id',
                  fill_color='Accent').add_to(map)

choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['SIG_ENG_NM'], lables=False)
)

map

In [ ]:
map = folium.Map((35.1795543,129.0756416), zoom_start=10)
choropleth = folium.Choropleth(json_busan,
                  data=gu_monthly_oil_max_price,
                  columns=['ENG_NM', '경유'],
                  key_on='feature.id',
                  fill_color='Accent').add_to(map)

choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['SIG_ENG_NM'], lables=False)
)

map